In [1]:
import fitz  # PyMuPDF
import os
import json
import uuid
from langchain.schema import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import pdfplumber


In [2]:
PDF_PATH = "test_doc.pdf"
OUTPUT_DIR = "pdf_chunks_v2"
EMBEDDING_MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
CHROMA_DB_DIR = "chroma_db_v2"


In [3]:
embedder = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL_NAME)


vector_store = Chroma(
    collection_name="test_vector_store",
    embedding_function=embedder,
    persist_directory=CHROMA_DB_DIR,
)

C:\Users\MSII\AppData\Local\Temp\ipykernel_2332\1329246236.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedder = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL_NAME)
d:\Working\Coding\ChunkingPDF\myEnv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\MSII\AppData\Local\Temp\ipykernel_2332\1329246236.py:4: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :cla

In [4]:
def save_image(doc, img_info, page_number):
    xref = img_info[0]
    base_image = doc.extract_image(xref)
    image_bytes = base_image["image"]
    img_ext = base_image["ext"]
    img_filename = f"page_{page_number}_img_{xref}.{img_ext}"
    img_path = os.path.join(OUTPUT_DIR, img_filename)
    with open(img_path, "wb") as f:
        f.write(image_bytes)
    return img_path

In [5]:
def extract_tables_from_page(pdf_path, page_number):
    tables = []
    with pdfplumber.open(pdf_path) as pdf:
        page = pdf.pages[page_number]
        extracted_tables = page.extract_tables()
        for table in extracted_tables:
            md_table = ""
            for row in table:
                md_table += "| " + " | ".join(cell or "" for cell in row) + " |\n"
            tables.append(md_table)
    return tables


In [6]:
def create_chunks_from_PDF_Page(blocks, images, page_number):
    all_chunks = []
    tables = extract_tables_from_page(PDF_PATH, page_number)
    tables_page = (" ").join(tables)
    current_chunk = {"text": "", "table": "", "image_path": None, "image_caption": None, "y0": None}
    last_y1 = 0

    for block in sorted(blocks, key=lambda b: b[1]):  # sort by y0
        x0, y0, x1, y1, text, _ = block
        text = text.strip()
        if not text:
            continue
        if current_chunk["y0"] is None:
            current_chunk["y0"] = y0
        if y0 - last_y1 > 40 and current_chunk["text"]:
            all_chunks.append(current_chunk)
            current_chunk = {"text": "", "table": "", "image_path": None, "image_caption": None, "y0": None}
        current_chunk["text"] += text + "\n"
        last_y1 = y1
    if current_chunk["text"]:
        all_chunks.append(current_chunk)
    if all_chunks:
        for image_path, image_y in images:
            closest_chunk = min(all_chunks, key=lambda c: abs(c.get("y0", 0) - image_y))
            closest_chunk["image_path"] = image_path
            closest_chunk["image_caption"] = f"Image from page {page_number}"
        for i in range(len(all_chunks)):
            if i == int(len(all_chunks)/2):
                print(f"Halfway through page {page_number}: {all_chunks[i]}")
                all_chunks[i]["table"] = tables_page
    return all_chunks






In [7]:
def proccess_pdf(pdf_path):
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    doc = fitz.open(pdf_path)
    all_chunks = []
    for page_number in range(len(doc)):
        page = doc[page_number]
        blocks = page.get_text("blocks")
        block_info = [
            (b[0], b[1], b[2], b[3], b[4], i) for i, b in enumerate(blocks)
        ]
        img_list = page.get_images(full=True)
        images = []
        for img in img_list:
            img_path = save_image(doc, img, page_number)
            y_pos = (page.rect.height / 2)
            images.append((img_path, y_pos))
        chunks = create_chunks_from_PDF_Page(block_info, images, page_number)
        for i, chunk in enumerate(chunks):
            chunk_id = f"page_{page_number}_chunk_{i}_{uuid.uuid4().hex[:6]}"
            chunk["id"] = chunk_id
            chunk["page"] = page_number
            all_chunks.append(chunk)
    with open(os.path.join(OUTPUT_DIR, "chunks.json"), "w", encoding="utf-8") as f:
        json.dump(all_chunks, f, indent=2, ensure_ascii=False)

    return all_chunks

        
    

In [8]:
def embed_and_store_chunks(chunks):
    docs = []
    for chunk in chunks:
        content = chunk["text"]
        metadata = {
                "chunk_id": chunk["id"],
                "page": chunk["page"],
                "table": chunk.get("table", ""),
        }
        # Chỉ thêm image_path nếu khác None
        if chunk.get("image_path") is not None:
            metadata["image_path"] = str(chunk["image_path"])

        docs.append(Document(page_content=content, metadata=metadata))
    vector_store.add_documents(docs)
    vector_store.persist()
    print("✅ Vectorstore saved to:", CHROMA_DB_DIR)


In [9]:
chunks = proccess_pdf(PDF_PATH)
embed_and_store_chunks(chunks)
print(f"✅ Total chunks: {len(chunks)}")

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


Halfway through page 0: {'text': 'CÔNG AN TP HẢI PHÒNG \nCÔNG AN QUẬN KIẾN AN\nCỘNG HÕA XÃ HỘI CHỦ NGHĨA VIỆT NAM\nĐộc lập – Tự do – Hạnh phúc\nKiến An, ngày 24 tháng 10 năm 2022\nTÀI LIỆU \nHƯỚNG DẪN CÀI ĐẶT ỨNG DỤNG VNEID, ĐĂNG KÝ VÀ KÍCH HOẠT TÀI\nKHOẢN ĐỊNH DANH ĐIỆN TỬ\nPHẦN I: TÓM TẮT QUY TRÌNH, CÁC BƯỚC THỰC HIỆN\nĐĂNG KÝ, KÍCH HOẠT TÀI KHOẢN MỨC ĐỘ 1 \nTHỦ TỤC ĐĂNG KÝ, KÍCH HOẠT TÀI KHOẢN MỨC ĐỘ 2\nChỉ áp dụng với người đã có thẻ CCCD gắn chíp điện tử\nChỉ áp dụng với người đã có thẻ CCCD gắn chíp điện tử\nBước 1:  Đến công an xã/phường nơi làm CCCD\nBước 1: Tải – cài đặt ứng dụng VNeID\nBước 2: Xuất trình CCCD gắn chíp, SĐT/email để bổ sung thông tin \nvào tài khoản\nBước 2: Đăng ký tài khoản trên ứng dụng VNeID\nBước 3: Đăng ký tài khoản định danh mức độ 1\nBước 3: Cán bộ nhập thông tin, chụp ảnh, lấy vân tay\nBước 4: Chụp/ tải ảnh chân dung\nBước 4: Chờ kết quả thông báo gửi đến SĐT đã đăng ký\nBước 5: Gửi đề nghị cấp, xác thực tài khoản định danh\nBước 5: Tải – cài đặt ứng 

CropBox missing from /Page, defaulting to MediaBox


Halfway through page 1: {'text': 'PHẦN III: DIỄN GIẢI CHI TIẾT BẰNG HÌNH ẢNH MỘT SỐ BƯỚC THỰC HIỆN\nLưu ý: Các thao tác thực hiện theo các bước hướng dẫn bạn có thể  truy cập trực tiếp vào trang chủ theo địa \nchỉ  https://vneid.gov.vn  bằng trình duyệt web có trên thiết bị di động hoặc máy tính của bạn (các trình duyệt web bạn đã cài đặt \ncó thể là: chrome; cốc cốc; fiefox ; safari… ) và thực hiện các bước hướng dẫn trang chủ vủa VNeID. Bạn có thể sử dụng thiết bị \ndi động thông minh đang sử dụng truy cập trực tiếp tới trang chủ của VNeID bằng cách click vào dòng chữ  https://vneid.gov.vn   \ntại tin nhắn SMS được hệ thống VNeID gửi tới số  điện thoại bạn đang sử dụng để truy cập tới trang chủ VNeID.\nTuy nhiên trong thực tế quá trình bạn thực hiện các thao tác trên máy tính hoặc trên thiết bị di động thông minh màn hình trên thiết bị \ncủa bạn sẽ hiển thị các hình ảnh minh họa cụ thể sau:\n1. Tải – Cài đặt ứng dụng VNeiD trên thiết bị di động thông minh\nQuy trình thực hiện tại: Bư

C:\Users\MSII\AppData\Local\Temp\ipykernel_2332\2339200621.py:16: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_store.persist()


In [12]:
query = "TÓM TẮT QUY TRÌNH, CÁC BƯỚC THỰC HIỆN ĐĂNG KÝ VÀ KÍCH HOẠT TÀI KHOẢN ĐỊNH DANH ĐIỆN TỬ"
ans = vector_store.similarity_search(query, k=1)

In [19]:
print(ans[0].page_content)
print(ans[0].metadata["table"])

CÔNG AN TP HẢI PHÒNG 
CÔNG AN QUẬN KIẾN AN
CỘNG HÕA XÃ HỘI CHỦ NGHĨA VIỆT NAM
Độc lập – Tự do – Hạnh phúc
Kiến An, ngày 24 tháng 10 năm 2022
TÀI LIỆU 
HƯỚNG DẪN CÀI ĐẶT ỨNG DỤNG VNEID, ĐĂNG KÝ VÀ KÍCH HOẠT TÀI
KHOẢN ĐỊNH DANH ĐIỆN TỬ
PHẦN I: TÓM TẮT QUY TRÌNH, CÁC BƯỚC THỰC HIỆN
ĐĂNG KÝ, KÍCH HOẠT TÀI KHOẢN MỨC ĐỘ 1 
THỦ TỤC ĐĂNG KÝ, KÍCH HOẠT TÀI KHOẢN MỨC ĐỘ 2
Chỉ áp dụng với người đã có thẻ CCCD gắn chíp điện tử
Chỉ áp dụng với người đã có thẻ CCCD gắn chíp điện tử
Bước 1:  Đến công an xã/phường nơi làm CCCD
Bước 1: Tải – cài đặt ứng dụng VNeID
Bước 2: Xuất trình CCCD gắn chíp, SĐT/email để bổ sung thông tin 
vào tài khoản
Bước 2: Đăng ký tài khoản trên ứng dụng VNeID
Bước 3: Đăng ký tài khoản định danh mức độ 1
Bước 3: Cán bộ nhập thông tin, chụp ảnh, lấy vân tay
Bước 4: Chụp/ tải ảnh chân dung
Bước 4: Chờ kết quả thông báo gửi đến SĐT đã đăng ký
Bước 5: Gửi đề nghị cấp, xác thực tài khoản định danh
Bước 5: Tải – cài đặt ứng dụng VNeiD (thực hiện khi thiết bị di động 
của bạn chưa 